In [1]:
import cv2
import numpy as np
import pyrealsense2 as rs

# Load YOLOv4 model and class labels (same as previous code)
net = cv2.dnn.readNetFromDarknet('yolov4.cfg', 'yolov4.weights')
with open('coco.names', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

In [2]:
# Function to perform object detection
def detect_objects(image):
    blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    layer_names = net.getLayerNames()
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]
    outputs = net.forward(output_layers)
    boxes = []
    confidences = []
    class_ids = []
    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * image.shape[1])
                center_y = int(detection[1] * image.shape[0])
                width = int(detection[2] * image.shape[1])
                height = int(detection[3] * image.shape[0])
                x = int(center_x - (width / 2))
                y = int(center_y - (height / 2))
                boxes.append([x, y, width, height])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    selected_boxes = []
    for i in indices:
        i = i[0]
        box = boxes[i]
        x, y, width, height = box
        label = classes[class_ids[i]]
        cv2.rectangle(image, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(image, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
        selected_boxes.append(box)
    return image, selected_boxes


In [3]:
# Configure RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start streaming
pipeline.start(config)

try:
    while True:
        # Wait for a new frame
        frames = pipeline.wait_for_frames()
        color_frame = frames.get_color_frame()

        if not color_frame:
            continue

        # Convert the color frame to a numpy array
        image = np.asanyarray(color_frame.get_data())

        # Check if the image array is valid
        if image is None or image.shape[0] == 0 or image.shape[1] == 0:
            continue

        # Detect objects in the frame
        output_frame, _ = detect_objects(image)

        # Display the output frame
        cv2.imshow('Object Detection', output_frame)

        # Break the loop on 'q' key press
        if cv2.waitKey(1) == ord('q'):
            break

except Exception as e:
    print(f"Error occurred: {e}")

finally:
    # Stop streaming and close windows
    pipeline.stop()
    cv2.destroyAllWindows()

Error occurred: invalid index to scalar variable.
